## Regex con dates.txt

El funcionamiento de esta aplicación se divide en las siguientes secciones:

1. Extracción de fechas mediante expresiones regulares.
2. Estandarización de texto y de formato.
3. Ordenamiento.

La extracción de fechas se hace con una expresión regular que identifica las posibles variaciones de escritura de fechas. Se encuentra en la variable 'regex'. 

La estandarización de texto y formato convierte a todas las fechas al formato día / mes / año, y en particular transforma todas las variaciones del texto a un valor numérico que se guarda en una lista. Por ejemplo: Octubre, 12, 1997 -> 12/10/1997 -> [12, 10, 1997]

El ordenamiento se hace primero por día, luego por mes y finalmente por año.

### Extracción usando Regex

In [1]:
import re
f = open(r'C:\Users\Frank\Desktop\PCIC\Segundo Semestre\dates.txt', 'r')
regex = '(([\d]{1,2}[\/-]+){2}([\d]{4}|[\d]{2}))|([\d]{2}[ ][\w]+([ ][\d]{4}|[ ][\d]{2}))|([\w]+[.| ]+[\d]+[, ]+[\d]{4})|(([A-Z][\w]){3,}[ |,]+[\d]{4})|(([\d]{1,2}\/)?[1|2]+[\d]{3})'
aux = f.readlines()

In [2]:
clean_list = []
for _ in aux:
    txt_aux = _.replace('\n', '')
    clean_list.append(txt_aux)

regex_list = [re.search(regex, _) for _ in clean_list]
regex_clean_list = [_.group() for _ in regex_list]

### Estandarización

In [3]:
# Filtra las fechas de formato de Estados Unidos (mm/dd/yy -> dd/mm/yy)
non_us_date = []
for _ in clean_list:
    if re.search('(([\d]{1,2}[\/-]+){2}([\d]{4}|[\d]{2}))', _):
        a = re.search('(([\d]{1,2}[\/-]+){2}([\d]{4}|[\d]{2}))', _).group()
        aux = re.split('/|-', a)
        aux[0], aux[1] = aux[1], aux[0]
        non_us_date.append(aux)

In [4]:
split_list = []
for _ in regex_clean_list:
    _ = re.sub('[ |-]', '/', _)
    _ = re.sub('[.|,]', '', _)
    split = _.split('/')
    split_list.append(split)

In [5]:
# Se estandariza para los casos en donde no se tienen las fechas completas. 
# Se agrega un 1 correspondiente tanto al mes como al día en los casos que hacen falta.
# 1994 -> 1, 1, 1994 (Correspondiente al 1 de Enero de 1994)

for _ in split_list:
    if len(_) == 2:
        _.insert(0, 1)
    if len(_) == 1:
        _.insert(0,1)
        _.insert(0,1)

In [6]:
# Se combinan las listas previamente obtenidas para tener consistencia del formato de fechas
for i in range(len(non_us_date)):
    split_list[i] = non_us_date[i]

In [7]:
# Diccionario auxiliar para reescribir nuestra información.
fechas = {
    1 : ['Jan', 'January'],
    2 : ['Feb', 'February'],
    3 : ['Mar', 'March'],
    4 : ['Apr', 'April'],
    5 : ['May'],
    6 : ['Jun', 'June'],
    7 : ['Jul', 'July'],
    8 : ['Aug', 'August'],
    9 : ['Sep', 'September'],
    10 : ['Oct', 'October'],
    11 : ['Nov', 'November'],
    12 : ['Dec', 'December']
}

def check_dict(value):
    """
    Verifica si un valor está en el diccionario fechas.
    
    value = lista[index]
    """
    for j in fechas:
        if value in fechas[j]:
            return True

def swap_month(value, dl):
    """
    Cambia los valores de día y mes cuando el mes está en formato texto
    
    value = list[index] ; Proveniente de la entrada que se quiere analizar.
    """
    if check_dict(value):
        dl[0], dl[1] = dl[1], dl[0]

def standard_19(value):
    """
    Estandariza los años 1900s donde solo se usa los dos años finales (99 -> 1999)
    """
    if len(value[2]) == 2:
        aux = '19'+value[2]
        value[2] = aux

def get_fechas_key(value):
    """
    Regresa el key de la fecha para poder ordenar la lista a partir fechas.
    """
    if check_dict(value[1]):
        for x, y in fechas.items():
            if value[1] in y:
                value[1] = x

def int_list(slist):
    """
    slist = split_list[index] ; 
    """
    lista = []
    for _ in slist:
        try:
            lista.append(int(_))
        except ValueError:
            lista.append(1)
    return lista

In [8]:
# Las funciones solo se aplican a una línea a la vez
# Se tiene que iterar una única vez sobre la lista para cambiar los valores.

# lista_chida es una lista donde se aplican todas las funciones previamente escritas.
# es una lista donde cada entrada corresponde a las fechas en formato: [int(dd), int(mm), int(yyyy)]

lista_chida = []
for _ in split_list:
    swap_month(_[0], _)
    standard_19(_)
    get_fechas_key(_)
    aux = int_list(_)
    if aux[1] > 12: # Corregimos un pequeño detalle que se nos pasó en la extracción usando ER.
        aux[1] = 1
    lista_chida.append(aux)

In [9]:
print(lista_chida[:5])
print(lista_chida[-5:])

[[25, 3, 1993], [18, 6, 1985], [8, 7, 1971], [27, 9, 1975], [6, 2, 1996]]
[[1, 1, 1979], [1, 1, 2006], [1, 1, 2008], [1, 1, 2005], [1, 1, 1980]]


### Ordenamiento

In [10]:
# Primero ordenamos por año
def sort_year(a):
    return a[2]
lista_chida.sort(key = sort_year, reverse = True)

In [11]:
# Luego se ordena por mes.
def sort_month(a):
    return a[1]

aux = []
for _ in lista_chida:
    aux.append(_[2])
auxset = list(set(aux))

full_list = []
for _ in auxset:
    aux = []
    for j in lista_chida:
        if j[2] == _:
            aux.append(j)
    aux.sort(key=sort_month)
    for _ in aux:
        full_list.append(_) # Con esto agregamos la lista ordenada por año y por meses.

# Ordenamiento por día.
for i in range(1, len(full_list)-1):
    if full_list[i][1] == full_list [i-1][1] and full_list[i][0] > full_list[i-1][0]:
        full_list[i], full_list[i-1] = full_list[i-1], full_list[i]

In [12]:
print(full_list[:5])
print(full_list[-5:])

[[10, 4, 1971], [18, 5, 1971], [11, 7, 1971], [8, 7, 1971], [12, 9, 1971]]
[[30, 5, 2016], [1, 5, 2016], [19, 10, 2016], [13, 10, 2016], [1, 11, 2016]]


In [13]:
print(f"Las entradas de la lista corresponden a fechas en el formato: dd/mm/yyyy")
print(f"Longitud de la lista final: {len(full_list)}")
print(f"Primera fecha: {full_list[0]}. Es decir el 10 de Abril de 1971.")
print(f"Última fecha: {full_list[-1]}. Es decir 1ro de Noviembre del 2016.")

Las entradas de la lista corresponden a fechas en el formato: dd/mm/yyyy
Longitud de la lista final: 500
Primera fecha: [10, 4, 1971]. Es decir el 10 de Abril de 1971.
Última fecha: [1, 11, 2016]. Es decir 1ro de Noviembre del 2016.
